# Monte Carlo Methods I: Random Numbers and Random Walk

Computational physics often begins with the notion that classical equations can perfectly predict the evolution of a system if one knows the initial conditions precisely.
However, many real-world processes involve an inherent level of randomness or produce outcomes so sensitive to initial conditions that they effectively appear random.
Radioactive decay, governed by quantum mechanics, is an example of truly non-deterministic behavior, whereas chaotic classical systems like the three-body problem can exhibit sensitive dependence on initial conditions.
As a result, introducing randomness into our computational methods is not only natural for modeling genuinely stochastic systems but also highly advantageous even in deterministic settings.

Monte Carlo methods, which rely on random sampling to estimate quantities of interest, illustrate how such randomness can be leveraged as a powerful tool in numerical analysis.
The phrase "Monte Carlo" alludes to the famous casino district in Monaco, highlighting the essence of "gambling" with random draws to tackle challenging problems.
Since their popularization in mid-twentieth-century nuclear research, Monte Carlo algorithms have found widespread use in evaluating high-dimensional integrals, simulating physical processes in statistical mechanics, performing error analysis in data-driven fields, and more.

![Monte Carlo](https://upload.wikimedia.org/wikipedia/commons/1/1b/Monaco_pano.jpg)

Although real physical systems may exhibit genuine randomness (as seen in quantum events), computer algorithms generate numbers through deterministic procedures.
These "pseudo-random" sequences are designed to mimic statistical randomness but are entirely reproducible given a specific initial seed.
In practice, a well-crafted pseudo-random number generator can satisfy the statistical requirements of scientific simulations, especially if it has a sufficiently large period and low correlations across many dimensions. Modern generators such as the Mersenne Twister, PCG, and Xorshift provide excellent performance in these regards, far surpassing the simpler linear congruential generators of older programming standards.

## Pseudo-Random Number Generators (PRNGs)

In computational physics, random draws are often produced by deterministic algorithms called pseudo-random number generators.
These generators start from an internal seed and use a mathematical rule to transform it repeatedly, producing an output sequence that ideally appears random.
Although each sequence is entirely predictable if one knows the seed and the algorithm, a good generator will avoid obvious patterns and pass stringent statistical tests.

### Congruential Generators

Early implementations of pseudo-random number generators were often based on linear congruential generators. The simplest form of this approach looks something like
\begin{align}
x_{n+1} = (a x_n + c) \mathrm{\ mod\ } m,
\end{align}
where $x_n$ represents the state at step $n$, and $a$, $c$, and $m$ are carefully chosen integers.

In [ ]:
# In order to understand the concept of a random number generator, let's implement one ourself.

mynext = 1

def myrand(): # NOT RECOMMENDED for real application.
    global mynext # by default, python does not recognize variable in a different scope.
    mynext = mynext * 1103515245 + 12345
    return (mynext//65536) % 32768

# This random number generator would generate integers in the domain [0, 32768).
# This random is usually provided to user by

MYRAND_MAX = 32768-1

# There are reasons for choosing the strange constants.  Take a look at
#     https://en.wikipedia.org/wiki/Linear_congruential_generator
# if you are interested.

In [ ]:
# Now, every time we run `rand()`, we will get a different number

myrand()

In [ ]:
# We may just print many of them at the same time:

Rs = [myrand() for i in range(100)]
print(Rs)

In [ ]:
# Sometime it is useful to make sure your random number sequence remains the same.
# In our case, you may notice that we can simply reset the `mynext` global variable to reset the sequence.
# The value you put in `mynext` is often called the "seed".

print('The following two lists are not the same:')
print([myrand() for i in range(10)])
print([myrand() for i in range(10)])

print('We may ensure that they are the same by "seeding" the random number generator with a fixed value:')

mynext = 1234
print([myrand() for i in range(10)])

mynext = 1234
print([myrand() for i in range(10)])

In [ ]:
# We may even plot the random numbers

from matplotlib import pyplot as plt

ll = [[myrand() for i in range(100)] for j in range(100)]
plt.imshow(ll)

This random number generator is very simple and is the *sample* implementation in many ANSI C libraries!
However, because how the standard was written, this create a lot problems.
* The standard only require `RAND_MAX` be at least 32767.
  If one want to evulate 1e6 points (which is pretty small, as we will see below), you will actually be evaluating the same 32768 points 30 times each!
* Some implementation "tried" to imporve the algorithm, e.g., swapping the lower and higher bytes.
  But these tricks sometime ruins the generator!
* Integrating high-dimension space is an important application of Monte Carlo methods.
  However, the congruential generators create correlation in k-space.

When programming in Python, we often take advantage of the built-in random module or NumPy's random subpackage, both of which default to high-quality generators (the Mersenne Twister for Python's default, and similarly robust methods in NumPy).
We can seed these generators manually to ensure reproducible research, or we may allow them to seed automatically for convenience.
While these generators generally suffice for undergraduate-level simulations, more intricate studies---particularly those involving very large datasets or extremely sensitive measurements---may require advanced or specialized libraries for best results.

### Python's Random Number Library

We start by trying python's random number library.
It is implemented in a very traditional way, which relies on a global seed and "procedure" programming style:

In [ ]:
import random as pyr

print(pyr)
pyr.seed(0)

print(pyr.random())           # return a random float in the range [0,1)
print(pyr.randrange(100))     # return a random int in the range [0, stop)
print(pyr.randint(a=0,b=99))  # return a random int in the range [a, b+1); alias for randrange(a, b+1)

### Numpy's Random Number Module

Next, we  Let's now try numpy's random number module.
It can also be used with a global seed.
However, for modern applications, it is often better to use object oriented (OO) approach, where the random number generator's state is stored in an object.
This allows one to create multiple random number generators in a single application.

In [ ]:
import numpy as np

npr = np.random.default_rng(seed=0)
print(npr)

print(npr.random())                               # return a random float in the range [0,1)
print(npr.integers(low=0,high=99))                # return a random int in the range [low, high)
print(npr.integers(low=0,high=99,endpoint=True))  # return a random int in the range [low, high] = [low, high+1) for int

### JAX's Random Number Module

Although numpy's OO approach is already an improvement, it can create ugly bugs for parallel algorithm.
Accelerated by GPU, JAX takes a functional programming apporach.
The random number generator's state is stored in a "key" object.
The key is then passed into a jax function to generate random numbers.

In [ ]:
from jax import random as jxr

key = jxr.key(seed=0)
print(key)

print(jxr.uniform(key))                           # return a random float in the range [0,1)
print(jxr.randint(key, (), minval=0, maxval=99))  # return a random int in the range [minval, maxval)

Note that, unlike `numpy`, the same key always return the same random number:

In [ ]:
print(jxr.uniform(key))
print(jxr.uniform(key))
print(jxr.uniform(key))
print(jxr.uniform(key))

If we need a new random number, we can use `split()` to generate new subkeys:

In [ ]:
key, subkey = jxr.split(key)
print(key)
print(subkey)
print(jxr.uniform(subkey))
print(jxr.uniform(subkey))

In [ ]:
key, subkey = jxr.split(key)
print(key)
print(subkey)
print(jxr.uniform(subkey))
print(jxr.uniform(subkey))

### Array of Random Numbers

For both numpy and JAX, it is simple to create an array of random numbers:

In [ ]:
ll2 = npr.random(size=(100,100))
ll3 = jxr.uniform(key, shape=(100,100))

In [ ]:
from matplotlib import pyplot as plt

fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(9,3))
ax0.imshow(ll)
ax1.imshow(ll2)
ax2.imshow(ll3)

## Drawing Random Numbers from Distributions

The random numbers we generated so far, including the ones from our own congruential generator, can be seen as a "random variable" draw from a **uniform distribution**.

In [ ]:
# We may plot the results of these random number generators

fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(9,3))
ax0.hist(npr.random(1000))
ax1.hist(npr.random(1000))
ax2.hist(npr.random(1000))

However, many physical phenomena and statistical models require samples from specific probability distributions.
These range from exponential and Poisson processes in radioactive decay to Maxwell–Boltzmann velocity distributions in statistical mechanics.
A variety of sampling or transformation techniques exist to convert uniform random numbers into draws from virtually any target distribution.

### Inverse Transform Method

The inverse transform method is a basic technique for generating random numbers drawn from a specified distribution.
Suppose we have a continuous random variable $X$ with probability density function $f(x)$ and corresponding cumulative distribution function $F(x)$.
The method begins by noting that if $U$ is a uniform random variable on the interval $[0, 1)$, then the random variable $X = F^{-1}(U)$ follows the distribution described by $f(x)$.
In other words, the inverse of the CDF effectively maps uniform deviates to the desired target distribution.

To illustrate the procedure, consider the exponential distribution with a rate parameter $\lambda$.
Its probability density function is
\begin{align}
f(x) = \lambda\,e^{-\lambda x}, \quad x \ge 0,
\end{align}
and the cumulative distribution function is
\begin{align}
F(x) = 1 - e^{-\lambda x}.
\end{align}
Because $F(x)$ is invertible, the inverse function is
\begin{align}
F^{-1}(u) = -\frac{1}{\lambda} \ln\bigl(1 - u\bigr).
\end{align}
Thus, if $u$ is drawn uniformly from $[0,1)$, then
\begin{align}
x = -\frac{1}{\lambda}\,\ln\bigl(1 - u\bigr)
\end{align}
is an exponentially distributed random variable with parameter $\lambda$.
One may equivalently use $\ln(u)$ instead of $\ln(1 - u)$ since $u$ is uniform over $[0,1)$.

In [ ]:
def inverse(u, lmbda, N=1000):
    return -np.log(1 - u) / lmbda

In [ ]:
# Parameters:
lmbda = 1.0
N     = 10000

# Generate samples via our inverse transform function:
u = np.random.rand(N)
x = inverse(u, lmbda)

# Generate samples via NumPy's built-in exponential:
x_np = np.random.exponential(scale=1/lmbda, size=N)

In [ ]:
# Plot histograms for comparison:
X = np.linspace(0, 10)
Y = lmbda * np.exp(-lmbda * X)

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(6,3))

ax0.hist(x,   bins=100, density=True, alpha=0.6, label='Inverse Transform')
ax0.plot(X, Y)
ax0.set_title('Inverse Transform Samples')
ax0.set_xlabel('x')
ax0.set_ylabel('Probability Density')

ax1.hist(x_np, bins=100, density=True, alpha=0.6, label='NumPy Exponential')
ax1.plot(X, Y)
ax1.set_title('NumPy Exponential Samples')
ax1.set_xlabel('x')
ax1.set_ylabel('Probability Density')

fig.tight_layout()

### Rejection Method

Sometimes, we need to sample from a probability distribution whose cumulative distribution function is difficult or impossible to invert analytically.
In these cases, the rejection method (also called the acceptance–rejection method) provides a flexible alternative.
The key is to find a simple "proposal" distribution that we can easily sample from but that encloses the shape of our target distribution.
If we sample points under the proposal distribution and then reject those that would exceed the desired probability density, the remaining accepted points follow the intended target distribution.

To set up the rejection method, we need three main ingredients:

1. A probability density function $f(x)$ for the target distribution, defined on some interval or possibly $\mathbb{R}$.
2. A proposal distribution $g(x)$ that we can sample from easily (e.g., a uniform or Gaussian), along with a constant $C$ such that
   \begin{align}
   f(x) \le C\,g(x) \quad \text{for all valid }x.
   \end{align}
   This inequality ensures the proposal distribution "covers" or "bounds" the entire shape of $f(x)$.
4. A procedure to accept or reject each random draw based on how likely it is to come from $f(x)$.

The algorithm proceeds as follows:
1. Generate a random value $X$ according to the proposal distribution $g(x)$.
2. Generate a uniform random number $U \in [0,1)$.
3. Accept $X$ if
   \begin{align}
   U \le \frac{f(X)}{C\,g(X)},
   \end{align}
   otherwise reject $X$ and repeat.